In [ ]:
pip install neurokit2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
import pandas as pd
import neurokit2 as nk
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# set directory containing CSV files
directory = '/content/drive/MyDrive/scenario_4/fold_1/train/physiology/'
# use glob to get a list of all CSV files in the directory, glob give full file path
csv_files = glob.glob("directory/'*.csv")
#os give only file name in current path, not full file path
csvname = os.listdir(directory)
tmp = 0
for filename in csvname:
  df = pd.read_csv(directory+filename)
  #1000 Hz is sample rate of physilogy data
  temp_ecg= nk.ecg_process(df.ecg,1000)
  temp_bvp= nk.ppg_process(df.bvp,1000)
  temp_rsp= nk.rsp_process(df.rsp,1000)
  temp_gsr= nk.eda_process(df.gsr,1000)

  temp_emg_coru= nk.emg_process(df.emg_coru,1000)
  temp_emg_coru[0].EMG_Amplitude*=100     

  temp_emg_zygo= nk.emg_process(df.emg_zygo,1000)
  temp_emg_zygo[0].EMG_Amplitude*=100 

  temp_emg_trap= nk.emg_process(df.emg_trap,1000)
  temp_emg_trap[0].EMG_Amplitude*=100   

  df.skt.fillna(method='pad')
  temp_skt=nk.ppg_process(df.skt,1000)
  temp_skt[0].PPG_Clean+=df.skt.mean() 

  #now take relvant features from each data, such as heart rate from ecg, and insert them into original datafile
  df.insert(2, "Heart_Rate",temp_ecg[0].ECG_Rate, True)
  df.insert(4, "BVP_clean",temp_bvp[0].PPG_Clean, True)
  df.insert(6, "GSR_clean_phasic",temp_gsr[0].EDA_Phasic, True)
  df.insert(8, "RR_clean",temp_rsp[0].RSP_Rate, True)
  df.insert(10, "skt_clean",temp_skt[0].PPG_Clean, True)
  df.insert(12, "zygo_clean",temp_emg_zygo[0].EMG_Amplitude, True)
  df.insert(14, "coru_clean",temp_emg_coru[0].EMG_Amplitude, True)
  df.insert(16, "trap_clean",temp_emg_trap[0].EMG_Amplitude, True)

  #save all data
  path2=f'/content/drive/MyDrive/scenario_4/fold_1/Clean/physiology/{filename}.csv'
  df.to_csv(path2)